# Chapter 4: Working with time series data

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42) # Life, the Universe, and Everything

Let's start by loading the CSV data.  We'll use numpy for this.

In [2]:
!pwd


/home/aardvark/dev/pytorch_notebook_experimentation/notebooks


In [13]:
bikes_numpy = np.loadtxt(
    "../data/external/bike_sharing/hour-fixed.csv",
    dtype=np.float32,
    delimiter=",",
    skiprows=1,
    converters={1: lambda x: float(x[8:10])}
)

bikes = torch.from_numpy(bikes_numpy)
bikes

tensor([[1.0000e+00, 1.0000e+00,  ..., 1.3000e+01, 1.6000e+01],
        [2.0000e+00, 1.0000e+00,  ..., 3.2000e+01, 4.0000e+01],
        ...,
        [1.7378e+04, 3.1000e+01,  ..., 4.8000e+01, 6.1000e+01],
        [1.7379e+04, 3.1000e+01,  ..., 3.7000e+01, 4.9000e+01]])

In [14]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

In [15]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape,daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

In [17]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

In [18]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)
first_day()

tensor([[  1,   1,   1,   0,   1,   0,   0,   6,   0,   1,   0,   0,   0,   0,
           3,  13,  16],
        [  2,   1,   1,   0,   1,   1,   0,   6,   0,   1,   0,   0,   0,   0,
           8,  32,  40],
        [  3,   1,   1,   0,   1,   2,   0,   6,   0,   1,   0,   0,   0,   0,
           5,  27,  32],
        [  4,   1,   1,   0,   1,   3,   0,   6,   0,   1,   0,   0,   0,   0,
           3,  10,  13],
        [  5,   1,   1,   0,   1,   4,   0,   6,   0,   1,   0,   0,   0,   0,
           0,   1,   1],
        [  6,   1,   1,   0,   1,   5,   0,   6,   0,   2,   0,   0,   0,   0,
           0,   1,   1],
        [  7,   1,   1,   0,   1,   6,   0,   6,   0,   1,   0,   0,   0,   0,
           2,   0,   2],
        [  8,   1,   1,   0,   1,   7,   0,   6,   0,   1,   0,   0,   0,   0,
           1,   2,   3],
        [  9,   1,   1,   0,   1,   8,   0,   6,   0,   1,   0,   0,   0,   0,
           1,   7,   8],
        [ 10,   1,   1,   0,   1,   9,   0,   6,   0,   1,   0, 